* 일변동률 레이블 달기: Up, Hold, Down
- 대신증권에서 일봉 주가데이터 가져오기
- 시작가(Open)와 종가(Close) 로 변동율 계산
- 상승 Up, 보합 Hold, 하락Down에 따라 U/H/D로 레이블
- DB에 종목 테이블명_il_rate_label 이름으로 테이블 저장

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
# !pip install JPype1
# !pip install konlpy
# !pip install pyMySQL
# !pip install -U finance-datareader

In [2]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [24]:
# -- get_table_list()-----------------------------------------------
# 모든 종목의 테이블명 추출
def get_table_list():
  # DB 연결
  connection = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 커서 생성
  cursor = connection.cursor()
  # 정보 스키마를 쿼리하여 테이블 이름 추출
  query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'DB이름'"
  # 쿼리 실행
  cursor.execute(query)
  # 결과 가져오기
  tables = cursor.fetchall()
  # 연결 종료
  cursor.close()
  connection.close()
  # 전체 테이블명 리스트 리턴
  return tables
# -- get_table_list(): END ---------------------------------------

# -- get_one_il_stock_price() ------------------------------------
# 한종목 일봉데이터 DB에서 가져오기
def get_one_il_stock_price(stock_code):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 종목 지정:  stock_code
  scode = stock_code
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT * FROM {scode}"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  print(scode)
  # DB close
  conn.close()
  # 데이터 리턴
  return result_df
# -- get_one_il_stock_price():END---------------------------------

# -- set_il_change_rate_label()-----------------------------------
# 일 변동률과 레이블 달기
def set_il_change_rate_label(table_name):
  # 종목 테이블명 (리스트 형태)
  table=table_name
  # 해당 종목 일봉 데이터 가져오기
  df_day = get_one_il_stock_price(table[0])
  # 문자열 date 컬럼을 datetime형식으로 바꿔서 datetime_date 컬럼에 넣기
  df_day['datetime_date']= pd.to_datetime(df_day['date'])
  # 2023년01월01일 이후 데이터만 가져오기
  df_day = df_day[df_day['datetime_date']>=pd.to_datetime('20230101')]
  # 일 변동율 계산해서 il_rate 컬럼에 넣기
  df_day['il_rate'] = ((df_day['close']-df_day['open'])/df_day['open'])*100
  # 일 변동율(%) 레이블 넣기: Up,Hold,Down
  df_day.loc[df_day['il_rate']>0,'il_label'] = 'U'
  df_day.loc[df_day['il_rate']==0,'il_label'] = 'H'
  df_day.loc[df_day['il_rate']<0,'il_label'] = 'D'
  df_result = df_day[['date','il_rate','il_label']]
  # 결과 df 리턴
  return df_result
# -- set_il_change_rate_label():END----------------------------------

# -- save_il_rate_label()--------------------------------------------
# 일 변동률과 레이블을 DB에 저장하기
def save_il_rate_label(df_il_rate_label,table):
  # 실행 확인용 화면 출력
  print(table[0][:-4])
  # DB 커넥션 생성
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소/DB이름'
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
  df_il_rate_label.to_sql(name=table[0][:-4]+'_il_rate_label', con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()
  # -- save_il_rate_label(): END ------------------------------------

In [ ]:
#========================================================
# 일 변동률과 레이블 달기 : 메인 실행
#--------------------------------------------------------
# 전체 종목 테이블 가져오기
table_list = get_table_list()
# 한 종목씩 처리하기
for table in table_list:
  # 실행 확인용 화면 출력: 해당 종목테이블명
  print(table[0][:-4])
  # 일별 변동률과 레이블 구하기
  df_il_rate_label = set_il_change_rate_label(table)
  # DB에 저장하기
  save_il_rate_label(df_il_rate_label,table)
#========================================================